<a href="https://colab.research.google.com/github/MohamedElSobkey/Deep-Neural-Networks/blob/main/Deep_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
data = pd.read_csv('/content/Airline_Delay_Cause.csv')

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data = data.drop(['carrier','carrier_name','airport','airport_name'],axis=1)
data

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.info()

In [ ]:
data['weather_delay'].min(),data['weather_delay'].max()

In [ ]:
#creating new column if the value more than 100 it will be 1 else it will be 0
data['WDCase'] = data['weather_delay'].apply(lambda x : 1 if x >100 else 0)

In [ ]:
data['WDCase'].value_counts()

In [ ]:
data

In [ ]:
X = data.drop(['WDCase'],axis = 1)
y =data['WDCase']


In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=44, shuffle =True)

print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)

In [ ]:
import tensorflow as tf
import keras

In [ ]:
KerasModel = keras.models.Sequential([
                # keras.layers.Input(shape=(17)),
            keras.layers.Dense(8,  activation = 'tanh'),
            # keras.layers.Dropout(0.1),
            keras.layers.Dense(128, activation = 'sigmoid'),
            # keras.layers.Dropout(0.3),
            keras.layers.Dense(64, activation = 'tanh'),
            keras.layers.Dense(32, activation = 'tanh'),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(1, activation = 'sigmoid')
            ])

In [ ]:
2,4,8,16,32,64,128,256,512 # prefered numbers of neurons to get high acc

In [ ]:
MyOptimizer = tf.keras.optimizers.experimental.AdamW(
    learning_rate=0.001,
    weight_decay=0.004,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name="AdamW")

In [ ]:
KerasModel.compile(optimizer =MyOptimizer,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = KerasModel.fit(X_train,y_train,
                         validation_data=(X_test,y_test),
                         epochs=100,
                         batch_size=10000,
                         verbose=1, # for show the result
                         callbacks=[tf.keras.callbacks.EarlyStopping( # early stopping
                                            patience=10, # wait for 10 epoch
                                            monitor='val_accuracy',#"val_loss",
                                            restore_best_weights=True)])

In [ ]:
print(KerasModel.summary())

In [ ]:
KerasModel.save('KerasModel.model')

In [ ]:
NewKerasModel = keras.models.load_model('KerasModel.model')

In [ ]:
X_test

In [ ]:
y_pred = NewKerasModel.predict(X_test)

In [ ]:
print('Prediction Shape is {}'.format(y_pred.shape))

In [ ]:
print('Prediction items are {}'.format(y_pred[:5]))

In [ ]:
y_test[:5]

In [ ]:
ModelLoss, ModelAccuracy = NewKerasModel.evaluate(X_test, y_test)
print('Model Loss is {}'.format(ModelLoss))
print('Model Accuracy is {}'.format(ModelAccuracy ))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:

len(y_test), len(y_pred)

In [ ]:
y_pred


In [ ]:
y_pred = [np.round(i[0]) for i in y_pred]
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
CM = confusion_matrix(y_test, y_pred)
print('Confusion Matrix is : \n', CM)

In [ ]:
# drawing confusion matrix
sns.heatmap(CM, center = True,cmap='Blues_r')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
ClassificationReport = classification_report(y_test,y_pred)
print('Classification Report is : \n', ClassificationReport )